# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
assessments = pd.read_csv("IUCN Plant Data_Bulk/assessments.csv")
taxonomy = pd.read_csv("IUCN Plant Data_Bulk/taxonomy.csv")
habitats = pd.read_csv("IUCN Plant Data_Bulk/habitats.csv")
countries = pd.read_csv("IUCN Plant Data_Bulk/countries.csv")
threats = pd.read_csv("IUCN Plant Data_Bulk/threats.csv")
usetrade = pd.read_csv("IUCN Plant Data_Bulk/usetrade.csv")

C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\2427298826.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  habitats = pd.read_csv("IUCN Plant Data_Bulk/habitats.csv")
C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\2427298826.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  threats = pd.read_csv("IUCN Plant Data_Bulk/threats.csv")


# Filter Data Deficient species

In [3]:
assessments_notDD = assessments.loc[assessments["redlistCategory"] != "Data Deficient"]

In [4]:
assessments_notDD["internalTaxonId"].count()

53657

# Fuse LR/LC & LC, LR/NT & NT, LR/CD & LC

In [5]:
assessments_notDD.replace({
        "Lower Risk/least concern": "Least Concern",
        "Lower Risk/near threatened": "Near Threatened",
        "Lower Risk/conservation dependent": "Least Concern",
        "Extinct in the Wild": "Extinct"
    }, inplace=True)
assessments_notDD["redlistCategory"].value_counts()

C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\2155290750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assessments_notDD.replace({


Least Concern            26657
Endangered                9400
Vulnerable                8959
Critically Endangered     4976
Near Threatened           3500
Extinct                    165
Name: redlistCategory, dtype: int64

In [6]:
assessments_notDD["internalTaxonId"].count()

53657

# assessments
Features: systems, realm, populationTrend
<br>
Target variable: redlistCategory

In [7]:
assessments_features = ["systems", "realm", "populationTrend"]
assessments_notDD.fillna("Unknown", inplace=True)

assessments_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
assessments_encoded = assessments_enc.fit_transform(assessments_notDD[assessments_features])
assessments_encoded = pd.DataFrame(assessments_encoded, index=assessments_notDD.index)
assessments_encoded.columns = assessments_enc.get_feature_names_out()
assessments_encoded["internalTaxonId"] = assessments_notDD["internalTaxonId"]
assessments_encoded

C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\288212443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assessments_notDD.fillna("Unknown", inplace=True)


,systems_Freshwater (=Inland waters),systems_Freshwater (=Inland waters)|Marine,systems_Marine,systems_Terrestrial,systems_Terrestrial|Freshwater (=Inland waters),systems_Terrestrial|Freshwater (=Inland waters)|Marine,systems_Terrestrial|Marine,systems_Unknown,realm_Afrotropical,realm_Afrotropical|Antarctic,...,realm_Neotropical|Palearctic,realm_Oceanian,realm_Oceanian|Palearctic,realm_Palearctic,realm_Unknown,populationTrend_Decreasing,populationTrend_Increasing,populationTrend_Stable,populationTrend_Unknown,internalTaxonId
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,133722
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,151198
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,151697
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,151700
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,151705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58338,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,130047059
58339,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,131552927
58340,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,144301060
58341,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37996


In [8]:
assessments_encoded["internalTaxonId"].count()

53657

In [9]:
len(set(assessments_encoded.index) & set(assessments_notDD.index))

53657

# taxonomy
Features: phylumName, className, orderName, familyName, genusName

In [10]:
taxonomy_features = ["phylumName", "className", "orderName", "familyName", "genusName"]

taxonomy_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
taxonomy_encoded = taxonomy_enc.fit_transform(taxonomy[taxonomy_features])
taxonomy_encoded = pd.DataFrame(taxonomy_encoded)
taxonomy_encoded.columns = taxonomy_enc.get_feature_names_out()
taxonomy_encoded["internalTaxonId"] = taxonomy["internalTaxonId"]
taxonomy_encoded

,phylumName_ANTHOCEROTOPHYTA,phylumName_BRYOPHYTA,phylumName_CHAROPHYTA,phylumName_CHLOROPHYTA,phylumName_MARCHANTIOPHYTA,phylumName_RHODOPHYTA,phylumName_TRACHEOPHYTA,className_ANDREAEOPSIDA,className_ANTHOCEROTOPSIDA,className_BRYOPSIDA,...,genusName_Zostera,genusName_Zygia,genusName_Zygocarpum,genusName_Zygogynum,genusName_Zygopetalum,genusName_Zygophlebia,genusName_Zygophyllum,genusName_Zygoruellia,genusName_Zygotritonia,internalTaxonId
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133722
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151198
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151697
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151700
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58338,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130047059
58339,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131552927
58340,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,144301060
58341,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37996


# habitats
Features: name

In [11]:
habitats.rename(columns={"name": "habitat_name"}, inplace=True)
habitats_features = ["habitat_name"]
habitats.fillna("Unknown", inplace=True)

habitats_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
habitats_encoded = habitats_enc.fit_transform(habitats[habitats_features])
habitats_encoded = pd.DataFrame(habitats_encoded)
habitats_encoded.columns = habitats_enc.get_feature_names_out()
habitats_encoded["internalTaxonId"] = habitats["internalTaxonId"]
habitats_encoded = habitats_encoded.groupby("internalTaxonId").max()
habitats_encoded

,habitat_name_Artificial/Aquatic - Aquaculture Ponds,"habitat_name_Artificial/Aquatic - Canals and Drainage Channels, Ditches",habitat_name_Artificial/Aquatic - Excavations (open),habitat_name_Artificial/Aquatic - Irrigated Land (includes irrigation channels),habitat_name_Artificial/Aquatic - Ponds (below 8ha),habitat_name_Artificial/Aquatic - Salt Exploitation Sites,habitat_name_Artificial/Aquatic - Seasonally Flooded Agricultural Land,habitat_name_Artificial/Aquatic - Wastewater Treatment Areas,habitat_name_Artificial/Aquatic - Water Storage Areas (over 8ha),habitat_name_Artificial/Marine - Mari/Brackishculture Ponds,...,habitat_name_Wetlands (inland) - Permanent Rivers/Streams/Creeks (includes waterfalls),"habitat_name_Wetlands (inland) - Permanent Saline, Brackish or Alkaline Lakes","habitat_name_Wetlands (inland) - Permanent Saline, Brackish or Alkaline Marshes/Pools",habitat_name_Wetlands (inland) - Seasonal/Intermittent Freshwater Lakes (over 8ha),habitat_name_Wetlands (inland) - Seasonal/Intermittent Freshwater Marshes/Pools (under 8ha),"habitat_name_Wetlands (inland) - Seasonal/Intermittent Saline, Brackish or Alkaline Lakes and Flats","habitat_name_Wetlands (inland) - Seasonal/Intermittent Saline, Brackish or Alkaline Marshes/Pools",habitat_name_Wetlands (inland) - Seasonal/Intermittent/Irregular Rivers/Streams/Creeks,habitat_name_Wetlands (inland) - Shrub Dominated Wetlands,habitat_name_Wetlands (inland) - Tundra Wetlands (incl. pools and temporary waters from snowmelt)
internalTaxonId,,,,,,,,,,,,,,,,,,,,,
30311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208195122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208498657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208945147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# countries
Features: name

In [12]:
countries.rename(columns={"name": "country_name"}, inplace=True)
countries_features = ["country_name"]
countries.fillna("Unknown", inplace=True)

countries_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
countries_encoded = countries_enc.fit_transform(countries[countries_features])
countries_encoded = pd.DataFrame(countries_encoded)
countries_encoded.columns = countries_enc.get_feature_names_out()
countries_encoded["internalTaxonId"] = countries["internalTaxonId"]
countries_encoded = countries_encoded.groupby("internalTaxonId").max()
countries_encoded

,country_name_Afghanistan,country_name_Albania,country_name_Algeria,country_name_American Samoa,country_name_Andorra,country_name_Angola,country_name_Anguilla,country_name_Antarctica,country_name_Antigua and Barbuda,country_name_Argentina,...,"country_name_Venezuela, Bolivarian Republic of",country_name_Viet Nam,"country_name_Virgin Islands, British","country_name_Virgin Islands, U.S.",country_name_Wallis and Futuna,country_name_Western Sahara,country_name_Yemen,country_name_Zambia,country_name_Zimbabwe,country_name_Åland Islands
internalTaxonId,,,,,,,,,,,,,,,,,,,,,
30310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30315,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208195122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208498657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208945147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# threats
Features: name, stressName

In [13]:
threats.rename(columns={"name": "threat_name"}, inplace=True)
threats_features = ["threat_name", "stressName"]
threats.fillna("Unknown", inplace=True)

threats_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
threats_encoded = threats_enc.fit_transform(threats[threats_features])
threats_encoded = pd.DataFrame(threats_encoded)
threats_encoded.columns = threats_enc.get_feature_names_out()
threats_encoded["internalTaxonId"] = threats["internalTaxonId"]
threats_encoded = threats_encoded.groupby("internalTaxonId").max()
threats_encoded

,threat_name_Abstraction of ground water (agricultural use),threat_name_Abstraction of ground water (commercial use),threat_name_Abstraction of ground water (domestic use),threat_name_Abstraction of ground water (unknown use),threat_name_Abstraction of surface water (agricultural use),threat_name_Abstraction of surface water (commercial use),threat_name_Abstraction of surface water (domestic use),threat_name_Abstraction of surface water (unknown use),threat_name_Acid rain,threat_name_Agro-industry farming,...,stressName_Species mortality|Species disturbance|Hybridisation|Inbreeding|Other,stressName_Species mortality|Species disturbance|Inbreeding,stressName_Species mortality|Species disturbance|Inbreeding|Reduced reproductive success|Other,stressName_Species mortality|Species disturbance|Loss of pollinator,stressName_Species mortality|Species disturbance|Loss of pollinator|Reduced reproductive success,stressName_Species mortality|Species disturbance|Loss of pollinator|Reduced reproductive success|Other,stressName_Species mortality|Species disturbance|Other,stressName_Species mortality|Species disturbance|Reduced reproductive success,stressName_Species mortality|Species disturbance|Reduced reproductive success|Other,stressName_Unknown
internalTaxonId,,,,,,,,,,,,,,,,,,,,,
30310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205445399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
205446122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
207984529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# usetrade
Features: name

In [14]:
usetrade.rename(columns={"name": "usetrade_name"}, inplace=True)
usetrade_features = ["usetrade_name"]
usetrade.fillna("Unknown", inplace=True)

usetrade_enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
usetrade_encoded = usetrade_enc.fit_transform(usetrade[usetrade_features])
usetrade_encoded = pd.DataFrame(usetrade_encoded)
usetrade_encoded.columns = usetrade_enc.get_feature_names_out()
usetrade_encoded["internalTaxonId"] = usetrade["internalTaxonId"]
usetrade_encoded = usetrade_encoded.groupby("internalTaxonId").max()
usetrade_encoded

,usetrade_name_Construction or structural materials,usetrade_name_Establishing ex-situ production *,usetrade_name_Fibre,usetrade_name_Food - animal,usetrade_name_Food - human,usetrade_name_Fuels,"usetrade_name_Handicrafts, jewellery, etc.",usetrade_name_Manufacturing chemicals,usetrade_name_Medicine - human & veterinary,usetrade_name_Other (free text),usetrade_name_Other chemicals,usetrade_name_Other household goods,"usetrade_name_Pets/display animals, horticulture",usetrade_name_Poisons,usetrade_name_Research,usetrade_name_Sport hunting/specimen collecting,usetrade_name_Unknown,"usetrade_name_Wearing apparel, accessories"
internalTaxonId,,,,,,,,,,,,,,,,,,
30311,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30318,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30319,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205445399,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207984529,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
208498657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Combining encoded features to one DataFrame

In [15]:
X_preprocessed = assessments_encoded
X_preprocessed = X_preprocessed.merge(taxonomy_encoded, how="left", on="internalTaxonId", suffixes=(None, "_taxonomy"))
X_preprocessed = X_preprocessed.merge(habitats_encoded, how="left", on="internalTaxonId", suffixes=(None, "_habitats"))
X_preprocessed = X_preprocessed.merge(countries_encoded, how="left", on="internalTaxonId", suffixes=(None, "_countries"))
X_preprocessed = X_preprocessed.merge(threats_encoded, how="left", on="internalTaxonId", suffixes=(None, "_threats"))
X_preprocessed = X_preprocessed.merge(usetrade_encoded, how="left", on="internalTaxonId", suffixes=(None, "_usetrade"))
print(X_preprocessed["internalTaxonId"].count())
X_preprocessed

53657


,systems_Freshwater (=Inland waters),systems_Freshwater (=Inland waters)|Marine,systems_Marine,systems_Terrestrial,systems_Terrestrial|Freshwater (=Inland waters),systems_Terrestrial|Freshwater (=Inland waters)|Marine,systems_Terrestrial|Marine,systems_Unknown,realm_Afrotropical,realm_Afrotropical|Antarctic,...,usetrade_name_Medicine - human & veterinary,usetrade_name_Other (free text),usetrade_name_Other chemicals,usetrade_name_Other household goods,"usetrade_name_Pets/display animals, horticulture",usetrade_name_Poisons,usetrade_name_Research,usetrade_name_Sport hunting/specimen collecting,usetrade_name_Unknown,"usetrade_name_Wearing apparel, accessories"
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53652,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
53653,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53654,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53655,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
y = assessments_notDD["redlistCategory"]
y_preprocessed = y.replace({
    "Least Concern": 0,
    "Near Threatened": 1,
    "Vulnerable": 2,
    "Endangered": 3,
    "Critically Endangered": 4,
    "Extinct": 5
})
y_preprocessed

0        3
1        3
2        3
3        4
4        3
        ..
58338    0
58339    0
58340    0
58341    3
58342    2
Name: redlistCategory, Length: 53657, dtype: int64

## Place entries with NaNs into the Unknown categories

In [17]:
unknown = [x for x in X_preprocessed.columns if "_Unknown" in x]
unknown

['systems_Unknown',
 'realm_Unknown',
 'populationTrend_Unknown',
 'habitat_name_Unknown',
 'stressName_Unknown',
 'usetrade_name_Unknown']

In [18]:
X_preprocessed_fillna = X_preprocessed.copy()
X_preprocessed_fillna[unknown].fillna(1, inplace=True)
X_preprocessed_fillna.fillna(0, inplace=True)
X_preprocessed_fillna

C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\1447774684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_preprocessed_fillna[unknown].fillna(1, inplace=True)


,systems_Freshwater (=Inland waters),systems_Freshwater (=Inland waters)|Marine,systems_Marine,systems_Terrestrial,systems_Terrestrial|Freshwater (=Inland waters),systems_Terrestrial|Freshwater (=Inland waters)|Marine,systems_Terrestrial|Marine,systems_Unknown,realm_Afrotropical,realm_Afrotropical|Antarctic,...,usetrade_name_Medicine - human & veterinary,usetrade_name_Other (free text),usetrade_name_Other chemicals,usetrade_name_Other household goods,"usetrade_name_Pets/display animals, horticulture",usetrade_name_Poisons,usetrade_name_Research,usetrade_name_Sport hunting/specimen collecting,usetrade_name_Unknown,"usetrade_name_Wearing apparel, accessories"
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53652,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
53653,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53654,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53655,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X_preprocessed_fillna.to_csv("X_preprocessed_fillna.csv", index=False)

y_preprocessed_fillna = pd.DataFrame()
y_preprocessed_fillna["redlistCategory"] = y_preprocessed
y_preprocessed_fillna["internalTaxonId"] = assessments_notDD["internalTaxonId"]
y_preprocessed_fillna.to_csv("y_preprocessed_fillna.csv", index=False)

## Drop entries with NaNs

In [20]:
X_preprocessed_dropna = X_preprocessed.dropna()
X_preprocessed_dropna

,systems_Freshwater (=Inland waters),systems_Freshwater (=Inland waters)|Marine,systems_Marine,systems_Terrestrial,systems_Terrestrial|Freshwater (=Inland waters),systems_Terrestrial|Freshwater (=Inland waters)|Marine,systems_Terrestrial|Marine,systems_Unknown,realm_Afrotropical,realm_Afrotropical|Antarctic,...,usetrade_name_Medicine - human & veterinary,usetrade_name_Other (free text),usetrade_name_Other chemicals,usetrade_name_Other household goods,"usetrade_name_Pets/display animals, horticulture",usetrade_name_Poisons,usetrade_name_Research,usetrade_name_Sport hunting/specimen collecting,usetrade_name_Unknown,"usetrade_name_Wearing apparel, accessories"
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53624,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
53627,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53629,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53655,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
y_preprocessed_dropna = pd.DataFrame()
y_preprocessed_dropna["internalTaxonId"] = X_preprocessed_dropna["internalTaxonId"]
y_preprocessed_dropna = y_preprocessed_dropna.merge(y_preprocessed_fillna, how="left", on="internalTaxonId")
y_preprocessed_dropna

,internalTaxonId,redlistCategory
0,133722,3
1,151198,3
2,151697,3
3,151705,3
4,151720,3
...,...,...
9434,128048676,2
9435,154363328,4
9436,138597081,2
9437,37996,3


In [22]:
X_preprocessed_dropna.to_csv("X_preprocessed_dropna.csv", index=False)
y_preprocessed_dropna.to_csv("y_preprocessed_dropna.csv", index=False)

# Feature selection
Number of features: 100
<br>
Test: Chi-square (chi2)

In [23]:
fillna_selector = SelectKBest(chi2, k=100)
X_preprocessed_fillna_100features = pd.DataFrame(fillna_selector.fit_transform(X_preprocessed_fillna.drop("internalTaxonId", axis=1), y_preprocessed_fillna["redlistCategory"]))
X_preprocessed_fillna_100features.columns = fillna_selector.get_feature_names_out()
X_preprocessed_fillna_100features["internalTaxonId"] = X_preprocessed_fillna["internalTaxonId"]
X_preprocessed_fillna_100features.to_csv("X_preprocessed_fillna_100features.csv", index=False)

In [24]:
dropna_selector = SelectKBest(chi2, k=100)
X_preprocessed_dropna_100features = pd.DataFrame(dropna_selector.fit_transform(X_preprocessed_dropna.drop("internalTaxonId", axis=1), y_preprocessed_dropna["redlistCategory"]), index=X_preprocessed_dropna.index)
X_preprocessed_dropna_100features.columns = dropna_selector.get_feature_names_out()
X_preprocessed_dropna_100features["internalTaxonId"] = X_preprocessed_dropna["internalTaxonId"]
X_preprocessed_dropna_100features.to_csv("X_preprocessed_dropna_100features.csv", index=False)

In [25]:
X_preprocessed_dropna_100features

,realm_Indomalayan,realm_Nearctic,populationTrend_Decreasing,populationTrend_Stable,populationTrend_Unknown,className_CYCADOPSIDA,orderName_CYCADALES,orderName_SOLANALES,familyName_SOLANACEAE,familyName_ZAMIACEAE,...,threat_name_Work & other activities,stressName_Ecosystem conversion|Ecosystem degradation|Indirect ecosystem effects|Species mortality|Competition,stressName_Ecosystem degradation|Species mortality|Competition|Reduced reproductive success,stressName_Indirect ecosystem effects|Inbreeding|Reduced reproductive success,stressName_Other,usetrade_name_Food - animal,usetrade_name_Food - human,usetrade_name_Medicine - human & veterinary,usetrade_name_Unknown,internalTaxonId
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133722
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151198
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151697
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151705
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,151720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53624,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128048676
53627,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154363328
53629,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,138597081
53655,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37996


# Data Split
20% for testing

In [26]:

#X_unsplit = X_preprocessed_dropna_100features.merge(y_preprocessed_dropna, how="left", on="internalTaxonId", suffixes=(None, "_redlistCategory"))
#X_train, X_test = train_test_split(X_unsplit, test_size=0.2, random_state=1)
#X_train

,realm_Indomalayan,realm_Nearctic,populationTrend_Decreasing,populationTrend_Stable,populationTrend_Unknown,className_CYCADOPSIDA,orderName_CYCADALES,orderName_SOLANALES,familyName_SOLANACEAE,familyName_ZAMIACEAE,...,stressName_Ecosystem conversion|Ecosystem degradation|Indirect ecosystem effects|Species mortality|Competition,stressName_Ecosystem degradation|Species mortality|Competition|Reduced reproductive success,stressName_Indirect ecosystem effects|Inbreeding|Reduced reproductive success,stressName_Other,usetrade_name_Food - animal,usetrade_name_Food - human,usetrade_name_Medicine - human & veterinary,usetrade_name_Unknown,internalTaxonId,redlistCategory
6377,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32927,0
1825,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15115789,3
4701,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115629519,2
2133,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22486412,0
5486,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121932826,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,62022627,2
7813,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193990,3
905,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38520,4
5192,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,152350,0


In [43]:

X_unsplit = X_preprocessed_fillna_100features.merge(y_preprocessed_fillna, how="left", on="internalTaxonId", suffixes=(None, "_redlistCategory"))
X_train, X_test = train_test_split(X_unsplit, test_size=0.2, random_state=1)
X_train

,systems_Terrestrial|Freshwater (=Inland waters),realm_Afrotropical,realm_Indomalayan,realm_Nearctic,realm_Oceanian,populationTrend_Decreasing,populationTrend_Stable,populationTrend_Unknown,familyName_CAMPANULACEAE,genusName_Brugmansia,...,stressName_Ecosystem conversion|Ecosystem degradation|Species mortality|Species disturbance,stressName_Ecosystem degradation,stressName_Ecosystem degradation|Competition,stressName_Ecosystem degradation|Species disturbance,stressName_Ecosystem degradation|Species mortality,stressName_Ecosystem degradation|Species mortality|Species disturbance,stressName_Unknown,usetrade_name_Medicine - human & veterinary,internalTaxonId,redlistCategory
44016,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154364530,4
32056,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136528469,0
1941,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,200262,2
14091,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19892511,0
13180,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13490715,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,181581311,0
32511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136816742,2
5192,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,199719,0
12172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45827,3


# Data Balancing
Using Synthetic Minority Oversampling TEchnique (SMOTE)

In [44]:
X_unbalanced = X_train.drop("redlistCategory", 1)
y_unbalanced = X_train["redlistCategory"]
counter = Counter(y_unbalanced)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=4, n=4005 (7.464%)
Class=0, n=21331 (39.754%)
Class=2, n=7159 (13.342%)
Class=1, n=2794 (5.207%)
Class=3, n=7511 (13.998%)
Class=5, n=125 (0.233%)


C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\6890099.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_unbalanced = X_train.drop("redlistCategory", 1)


In [45]:
oversampler = SMOTE(k_neighbors=2)
X_sm, y_sm = oversampler.fit_resample (X_train.drop("redlistCategory",1), X_train["redlistCategory"])
X_sm

C:\Users\rayno\AppData\Local\Temp\ipykernel_4200\2352072801.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_sm, y_sm = oversampler.fit_resample (X_train.drop("redlistCategory",1), X_train["redlistCategory"])


,systems_Terrestrial|Freshwater (=Inland waters),realm_Afrotropical,realm_Indomalayan,realm_Nearctic,realm_Oceanian,populationTrend_Decreasing,populationTrend_Stable,populationTrend_Unknown,familyName_CAMPANULACEAE,genusName_Brugmansia,...,stressName_Ecosystem conversion|Ecosystem degradation|Species mortality,stressName_Ecosystem conversion|Ecosystem degradation|Species mortality|Species disturbance,stressName_Ecosystem degradation,stressName_Ecosystem degradation|Competition,stressName_Ecosystem degradation|Species disturbance,stressName_Ecosystem degradation|Species mortality,stressName_Ecosystem degradation|Species mortality|Species disturbance,stressName_Unknown,usetrade_name_Medicine - human & veterinary,internalTaxonId
0,0.0,0.00000,1.00000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,154364530
1,0.0,1.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,136528469
2,0.0,0.00000,0.00000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,200262
3,0.0,0.00000,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,19892511
4,0.0,1.00000,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,13490715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127981,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.310314,0.0,62798343
127982,0.0,0.59321,0.40679,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,37840
127983,0.0,1.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,45004
127984,0.0,1.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,41833


In [46]:
X_sm.to_csv("X_sm_100features.csv")
y_sm.to_csv("y_sm_100features.csv")

In [47]:
counter = Counter(y_sm)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=4, n=21331 (39.754%)
Class=0, n=21331 (39.754%)
Class=2, n=21331 (39.754%)
Class=1, n=21331 (39.754%)
Class=3, n=21331 (39.754%)
Class=5, n=21331 (39.754%)
